In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the model and tokenizer
model_name = "deepseek-ai/DeepSeek-Prover-V1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up the pipeline for text generation
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def test_prompt(prompt):
    """
    Test the model with a given prompt and print the output.
    """
    print(f"Testing prompt:\n{prompt}\n")
    output = generator(prompt, max_length=100, num_return_sequences=1, do_sample=True)
    generated_text = output[0]["generated_text"]
    print(f"Generated Output:\n{generated_text}\n")
    return generated_text

# Sample prompts for testing
prompts = [
    "Please provide the single tactic needed to complete the following step in a Lean proof. Make sure the tactic is enclosed in a Markdown code block using triple backticks (` ``` `). Output only the tactic and nothing else.",
    "Output only the tactic needed for the proof step. Enclose the tactic in a code block.",
    "Provide the tactic necessary for the proof step in a code block format.",
]

# Run tests on all prompts
for prompt in prompts:
    test_prompt(prompt)

In [1]:
import re

def extract_tactic_parser(completion: str, prompt_template: str) -> str:
    resulting_tactic = []
    # Regular expression to find text within triple backticks
    match_options = [
        # re.search(r'```(.*?)```', completion, re.DOTALL),
        re.search(r'/-(.*?)-/', completion, re.DOTALL)
    ]
    
    for match in match_options:
        if match:
            resulting_tactic.append(match.group(1).strip())
        else:
            # Fallback to find text within single backticks if no triple backticks
            match = re.search(r'`([^`]*)`', completion)
            if match:
                resulting_tactic.append(match.group(1).strip())
        # If no code block is found, return a fallback message or handle it as needed
        return "Tactic not found in expected format."



LEAN4_DEFAULT_HEADER = "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n"

def cot_prompt(data):
    return "Complete the following Lean 4 code with explanatory comments preceding each line of code:\n\n```lean4\n{header}{informal_prefix}{formal_statement}".format(
        header=data.get('header', LEAN4_DEFAULT_HEADER),
        informal_prefix=data.get('informal_prefix', str()),
        formal_statement=data['formal_statement'],
    )

def post_process_output(output):
    _find_idx = output.find("```")
    return output[:_find_idx] if _find_idx >= 0 else output


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from collections import defaultdict

tokenizer_id = "deepseek-ai/DeepSeek-Prover-V1"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
prompt_1 = "Suggest a next tactic given the following Lean 4 tactic state\n```lean4\n-- tactic state\n/-\ncase h.a\n\u03b1 : Type u\n\u03b2 : Type v\nl\u271d l\u2081 l\u2082 : List \u03b1\nr : \u03b1 \u2192 \u03b1 \u2192 Prop\na b : \u03b1\ninst\u271d : DecidableEq \u03b1\nl : List \u03b1\nhl : l.Nodup\ni : Fin l.length\nn\u271d : \u2115\na\u271d : \u03b1\n\u22a2 a\u271d \u2208 (l.erase (l.get i))[n\u271d]? \u2194 a\u271d \u2208 (l.eraseIdx \u2191i)[n\u271d]?\n-/\n-- next tactic\n"
prompt_2 = "Suggest a next tactic given the following Lean 4 tactic state\n```lean4\n-- tactic state\n/-\ncase cons.mk\n\u03b1 : Type u\n\u03b2 : Type v\nl l\u2081 l\u2082 : List \u03b1\nr : \u03b1 \u2192 \u03b1 \u2192 Prop\na b : \u03b1\ninst\u271d : DecidableEq \u03b1\nhead\u271d : \u03b1\ntail\u271d : List \u03b1\nval\u271d : \u2115\nisLt\u271d : val\u271d < (head\u271d :: tail\u271d).length\nhl : head\u271d \u2209 tail\u271d \u2227 tail\u271d.Nodup\n\u22a2 (head\u271d :: tail\u271d).erase (head\u271d :: tail\u271d)[val\u271d] = (head\u271d :: tail\u271d).eraseIdx val\u271d\n-/\n-- next tactic\n"
prompt_3 = "Suggest a next tactic given the following Lean 4 tactic state\n```lean4\n-- tactic state\n/-\nx y : \u211d\nhx : 0 < x\n\u22a2 (if h : x = 0 then 0\n    else\n      (StrictMono.orderIsoOfSurjective (codRestrict rexp (Real.lt 0) exp_pos) expOrderIso.proof_1\n            expOrderIso.proof_2).symm\n        \u27e8|x|, \u22ef\u27e9) =\n    (StrictMono.orderIsoOfSurjective (codRestrict rexp (Real.lt 0) exp_pos) expOrderIso.proof_1\n          expOrderIso.proof_2).symm\n      \u27e8x, hx\u27e9\n-/\n-- next tactic\n"
prompt_4 = "Suggest a next tactic given the following Lean 4 tactic state\n```lean4\n-- tactic state\n/-\ncase mk\n\u03b1 : Type u_1\ninst\u271d\u00b9 : CompleteLattice \u03b1\ninst\u271d : IsCompactlyGenerated \u03b1\nS : Set \u03b1\nh : sSup S = \u22a4\nisAtom\u271d : \u2200 I \u2208 S, IsAtom I\nfinitelyAtomistic\u271d : \u2200 (s : Finset \u03b1) (a : \u03b1), \u2191s \u2286 S \u2192 IsCompactElement a \u2192 a \u2264 s.sup id \u2192 \u2203 t \u2286 s, a = t.sup id\n\u22a2 ComplementedLattice \u03b1\n-/\n-- next tactic\n"
tactic0 = "exact fun \u03b9 s h => \u27e8range s, by simpa only [iSup_range] using h\u27e9\n"

state_only = "\ncase h.a\n\u03b1 : Type u\n\u03b2 : Type v\nl\u271d l\u2081 l\u2082 : List \u03b1\nr : \u03b1 \u2192 \u03b1 \u2192 Prop\na b : \u03b1\ninst\u271d : DecidableEq \u03b1\nl : List \u03b1\nhl : l.Nodup\ni : Fin l.length\nn\u271d : \u2115\na\u271d : \u03b1\n\u22a2 a\u271d \u2208 (l.erase (l.get i))[n\u271d]? \u2194 a\u271d \u2208 (l.eraseIdx \u2191i)[n\u271d]?\n"

def count_tokens(text):
    return len(tokenizer.tokenize(text))

# print(count_tokens(prompt_1))
# print(count_tokens(prompt_2))
# print(count_tokens(prompt_3))
# print(count_tokens(prompt_4))


prompt = "Suggest a next tactic given the following Lean 4 tactic state\n```lean4\n-- tactic state\n/-\ncase h.a\n\u03b1 : Type u\n\u03b2 : Type v\nl\u271d l\u2081 l\u2082 : List \u03b1\nr : \u03b1 \u2192 \u03b1 \u2192 Prop\na b : \u03b1\ninst\u271d : DecidableEq \u03b1\nl : List \u03b1\nhl : l.Nodup\ni : Fin l.length\nn\u271d : \u2115\na\u271d : \u03b1\n\u22a2 a\u271d \u2208 (l.erase (l.get i))[n\u271d]? \u2194 a\u271d \u2208 (l.eraseIdx \u2191i)[n\u271d]?\n-/\n-- next tactic\n"
tactic = "Suggest a next tactic given the following Lean 4 tactic state\n```lean4\n-- tactic state\n/-\ncase h.a\n\u03b1 : Type u\n\u03b2 : Type v\nl\u271d l\u2081 l\u2082 : List \u03b1\nr : \u03b1 \u2192 \u03b1 \u2192 Prop\na b : \u03b1\ninst\u271d : DecidableEq \u03b1\nl : List \u03b1\nhl : l.Nodup\ni : Fin l.length\nn\u271d : \u2115\na\u271d : \u03b1\n\u22a2 a\u271d \u2208 (l.erase (l.get i))[n\u271d] \u2194 a\u271d \u2208 l[n\u271d]\n-/\n-- next tactic\nrw [mem_of"


with open("/home/vincentzhu/gfn_ntp/data/parsing_differences_280.json") as f:
    data280 = json.load(f)

bad = defaultdict(list)
for k, v in data280.items():
    for tac in v:
        if tac.startswith("Suggest a next tactic given the following Lean 4 tactic state"):
            bad[k].append(tac)


In [10]:
for k, v in bad.items():
    print(count_tokens(k))
    print(count_tokens(v[0]))

161
159
184
159
182
159
166
159


In [ ]:
for j in range(num_samples):
    t0 = remove_marks(raw_output_text[j])
    t1 = ""
    t2 = remove_marks(raw_output_text[j])
    t3 = ""
    if self.decoder_only and t0.startswith(state):
        t1 = t0
        # skip prompt
        t0 = t0[len(state):]
        t1 = t1[len(state):]
        # end at next newline
        next_newline = t0.find("\n")
        if next_newline != -1:
            t0 = t0[:next_newline]
            logger.info(t0)
        next_newline = t1.find("\n```")
        if next_newline != -1:
            t1 = t1[:next_newline]
            logger.info(t1)
    if self.decoder_only and ("-- next tactic\n" in t2):
        t3 = t2
        # skip prompt
        t2 = t2.split("-- next tactic\n")[1]
        t3 = t3.split("-- next tactic\n")[1]
        # end at next newline
        next_newline = t2.find("\n")
        if next_newline != -1:
            t2 = t2[:next_newline]
            logger.info(t2)
        next_newline = t3.find("\n```")
        if next_newline != -1:
            t3 = t3[:next_newline]
            logger.info(t3)
    
    if len({t0, t1, t2, t3}) > 1: # Some of the tactics are different
        parsing_errors = []
        parsing_errors.append(t0)
        parsing_errors.append(t1)
        parsing_errors.append(t2)
        parsing_errors.append(t3)
        new_data = {
            state: parsing_errors
        }
        
        save_parsing_differences = "data/parsing_differences_280.json"
        try:
            with open(save_parsing_differences, 'r') as file:
                existing_data = json.load(file)  # Load the current data
        except FileNotFoundError:
            # If the file doesn't exist, start with an empty dictionary
            existing_data = {}

        existing_data.update(new_data)
        with open(save_parsing_differences, 'w') as file:
            json.dump(existing_data, file, indent=4)


    # if  "Suggest a next tactic given the following Lean 4 tactic state" in any(t0, t1, t2, t3):        
    #     print(f"[bold red]PROMPT STILL IN A TACTIC[/bold red]")
    #     time.sleep(60)
    #     contains_prompt = []
    #     contains_prompt.append(t0)
    #     contains_prompt.append(t1)
    #     contains_prompt.append(t2)
    #     contains_prompt.append(t3)
    #     new_data = {
    #         state: contains_prompt
    #     }
        
    #     contains_prompt_save = "data/contains_prompt.json"
    #     try:
    #         with open(contains_prompt_save, 'r') as file:
    #             existing_data = json.load(file)  # Load the current data
    #     except FileNotFoundError:
    #         # If the file doesn't exist, start with an empty dictionary
    #         existing_data = {}

    #     existing_data.update(new_data)
    #     with open(contains_prompt_save, 'w') as file:
    #         json.dump(existing_data, file, indent=4)
    